In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm





In [ ]:
def load_calibration_matrix(calib_file):
    with open(calib_file) as f:
        lines = f.readlines()
        calib_matrix = np.array([list(map(float, line.strip().split())) for line in lines])
        return calib_matrix

In [ ]:

def downscale_image(image, factor):
    for _ in range(int(factor / 2)):
        image = cv2.pyrDown(image)
    return image


In [ ]:

def find_common_points(image_points_1, image_points_2):
    common_points_1 = []
    common_points_2 = []

    for i, point_1 in enumerate(image_points_1):
        for j, point_2 in enumerate(image_points_2):
            if np.array_equal(point_1, point_2):
                common_points_1.append(i)
                common_points_2.append(j)

    return np.array(common_points_1), np.array(common_points_2)





In [ ]:
def process_images(img_dir, downscale_factor=2.0, enable_bundle_adjustment=False):
    image_list = sorted([image for image in os.listdir(img_dir) if image.lower().endswith('.jpg') or image.lower().endswith('.png')])

    if len(image_list) < 2:
        print("Insufficient images for processing.")
        return

    calib_matrix = load_calibration_matrix(os.path.join(img_dir, 'calib.txt'))
    K = calib_matrix / downscale_factor

    feature_params = dict(maxCorners=100,
                          qualityLevel=0.3,
                          minDistance=7,
                          blockSize=7)

    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    color = np.random.randint(0, 255, (100, 3))

    prev_image = cv2.imread(os.path.join(img_dir, image_list[0]))
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

    total_points = np.zeros((1, 3))
    total_colors = np.zeros((1, 3))

    pose_array = []

    for i in tqdm(range(1, len(image_list))):
        image = cv2.imread(os.path.join(img_dir, image_list[i]))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, p0, None, **lk_params)
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        E, _ = cv2.findEssentialMat(good_old, good_new, K, cv2.RANSAC, 0.999, 1.0, None)
        _, R, t, _ = cv2.recoverPose(E, good_old, good_new, K)

        if i != 1:
            F, _ = cv2.findFundamentalMat(good_old, good_new, cv2.RANSAC)
            _, H1, H2 = cv2.stereoRectifyUncalibrated(good_old, good_new, F, (prev_gray.shape[1], prev_gray.shape[0]))

            points, _ = cv2.correctMatches(F, good_old.reshape(-1, 1, 2), good_new.reshape(-1, 1, 2))
            points = points.squeeze()

            common_points_1, common_points_2 = find_common_points(good_old, points)
            common_points_2 = common_points_2[:common_points_1.shape[0]]

            good_new = good_new[common_points_2]
            good_old = good_old[common_points_1]

        _, points_3d = cv2.triangulatePoints(np.eye(3, 4), np.hstack((R, t)), good_old.T, good_new.T)
        points_3d = points_3d / points_3d[3]

        if enable_bundle_adjustment and i != 1:
            _, points_3d = cv2.correctMatches(F, good_old.reshape(-1, 1, 2), points_3d[:3].T)
            points_3d = points_3d.squeeze()

        _, rot_matrix, tran_matrix, _ = cv2.recoverPose(E, good_old, good_new, K)

        if enable_bundle_adjustment:
            pose_array.extend([pose.ravel() for pose in np.hstack((rot_matrix, tran_matrix.reshape(-1, 1)))])

        _, rot_matrix, tran_matrix, _ = cv2.solvePnPRansac(points_3d, good_new, K, None)
        transform_matrix = np.hstack((rot_matrix, tran_matrix))

        if enable_bundle_adjustment:
            pose_array.extend([pose.ravel() for pose in transform_matrix])

        pose_array.extend([pose.ravel() for pose in transform_matrix])

        _, rot_matrix, tran_matrix, _ = cv2.solvePnPRansac(points_3d, good_new, K, None)
        transform_matrix = np.hstack((rot_matrix, tran_matrix))

        if enable_bundle_adjustment:
            pose_array.extend([pose.ravel() for pose in transform_matrix])

        pose_array.extend([pose.ravel() for pose in transform_matrix])

        if enable_bundle_adjustment and i != 1:
            opt = cv2.fisheye_CALIB_RATIONAL_MODEL
            _, points_3d = cv2.fisheye.triangulatePoints(H1, H2, good_old.T, good_new.T)
            points_3d = points_3d / points_3d[3]

            _, points_3d = cv2.correctMatches(F, good_old.reshape(-1, 1, 2), points_3d[:3].T)
            points_3d = points_3d.squeeze()

            points_3d, _, _ = cv2.fisheye.calibrate([points_3d], None, (image.shape[1], image.shape[0]), K, None, None, None, flags=opt)
            points_3d = points_3d.squeeze()

        pose_array.extend([pose.ravel() for pose in np.hstack((rot_matrix, tran_matrix.reshape(-1, 1)))])
        total_points = np.vstack((total_points, points_3d))

        points_left = np.array(good_new, dtype=np.int32)
        color_vector = np.array([image[l[1], l[0]] for l in points_left])
        total_colors = np.vstack((total_colors, color_vector))

        prev_gray = gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    print("Processing completed.")

    if enable_bundle_adjustment:
        K = K / downscale_factor
        ret, K, D, rvecs, tvecs = cv2.calibrateCamera(total_points.reshape(-1, 1, 3), total_colors.reshape(-1, 1, 3), (image.shape[1], image.shape[0]), K, None, None)
        print("Bundle adjustment completed.")

    return total_points, total_colors, pose_array

In [ ]:

if __name__ == '__main__':
    data_dir = "data\\set1"
    downscale_factor = 2.0
    enable_bundle_adjustment = True

    total_points, total_colors, pose_array = process_images(data_dir, downscale_factor, enable_bundle_adjustment)

    print("Printing to .ply file")
    print(total_points.shape, total_colors.shape)

    out_points = total_points * 200
    out_colors = total_colors

    mean = np.mean(out_points, axis=0)
    scaled_verts = out_points - mean
    dist = np.sqrt(np.sum(scaled_verts ** 2, axis=1))
    indx = np.where(dist < np.mean(dist) + 300)
    verts = np.hstack([out_points[indx], out_colors[indx]])

    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''

    ply_path = os.path.join(data_dir, 'set1.ply')
    with open(ply_path, 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')